In [ ]:
from __future__ import annotations

from pathlib import Path

from typing_extensions import Annotated

import pytask
from pytask import DirectoryNode, ExitCode, task

In [ ]:
def task_create_files() -> Annotated[None, DirectoryNode(pattern="[ab].txt")]:
    path = Path()
    path.joinpath("a.txt").write_text("Hello, ")
    path.joinpath("b.txt").write_text("World!")


@task(after=task_create_files, is_generator=True)
def task_generator_copy_files(
    paths: Annotated[list[Path], DirectoryNode(pattern="[ab].txt")]
):
    for path in paths:

        @task
        def task_copy(
            path: Path = path,
        ) -> Annotated[str, path.with_name(path.stem + "-copy.txt")]:
            return path.read_text()

In [ ]:
session = pytask.build(tasks=[task_create_files, task_generator_copy_files])
assert session.exit_code == ExitCode.OK